In [2]:
import pandas as pandas

df = pandas.read_csv('./dataset/games.csv')

df.head()

,id,kills,kd,dmg,deaths,hsPct
1992934777753343208,11,1.64,'4,101',0,0%
1992934777753343208,7,1.20,'1,210',0,0%
1992934777753343208,6,1.21,'2,680',0,33%
1992934777753343208,1,1.03,'1,780',1,100%
1992934777753343208,12,4.98,'5,693',5,25%
